<a href="https://colab.research.google.com/github/Krankile/npmf/blob/main/notebooks/inner_join_fundamentals_and_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Kernel setup

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
%%capture
!pip install wandb
!git clone https://github.com/Krankile/npmf.git

In [4]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##General setup

In [11]:
%%capture
!cd npmf && git pull

import os
from collections import defaultdict
from collections import Counter
from datetime import datetime
from operator import itemgetter

import numpy as np
from numpy.ma.core import outerproduct
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

import wandb as wb

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from npmf.utils.colors import main, main2, main3
from npmf.utils.wandb import get_dataset, put_dataset
from npmf.utils.eikon import column_mapping 

In [6]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[main, main2, main3, "black"])
mpl.rcParams['figure.figsize'] = (6, 4)  # (6, 4) is default and used in the paper

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [8]:
np.random.seed(420)

# Data processing

## Collect data from stock and fundamental and find intersection of unique tickers

In [9]:
stock_df = get_dataset("stock-data-clean:latest", "master-test")
fundamental_df = get_dataset("fundamental-data-clean:latest", "master-test")
meta_df = get_dataset("oil-meta-data:latest", "master-test")

wandb: Currently logged in as: krankile (use `wandb login --relogin` to force relogin)


wandb: Downloading large artifact stock-data-clean:latest, 126.38MB. 1 files... Done. 0:0:0


In [12]:
unique_fundamental = set(fundamental_df["Instrument"].unique())
unique_stock = set(stock_df["ticker"].unique())

full_data_companies = unique_fundamental & unique_stock

## Inspect the raw data

To know what we're working with

In [17]:
stock_df.head()

,ticker,date,market_cap,close_price,currency
0,000096.SZ,2000-07-24,461931447.619507,0.87487,USD
1,000096.SZ,2000-07-25,487445197.289756,0.923192,USD
2,000096.SZ,2000-07-26,476291125.175129,0.902067,USD
3,000096.SZ,2000-07-27,481888565.458432,0.912668,USD
4,000096.SZ,2000-07-28,490252916.878034,0.928509,USD


In [18]:
fundamental_df.head()

,index,ticker,date,period_end_date,announce_date,revenue,gross_profit,ebitda,ebit,net_income,...,total_current_assets,total_liabilites,total_current_liabilities,long_term_debt_p_assets,short_term_debt_p_assets,fcf,gross_profit_p,ebitda_p,ebit_p,net_income_p
0,1,OMVV.VI,2000-06-30T00:00:00Z,2000-06-30,2000-10-23,1591395023.28372,230190745.555143,181045434.200878,105530002.190288,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.144647,0.113765,0.066313,NaN
1,3,OMVV.VI,2000-12-31T00:00:00Z,2000-12-31,2001-04-30,1938098647.94837,372273990.672257,228378009.139304,132398360.578508,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.192082,0.117836,0.068314,NaN
2,4,OMVV.VI,2001-03-31T00:00:00Z,2001-03-31,2001-11-15,1675754784.0278,270127135.374168,246253937.336036,174937923.894256,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.161197,0.146951,0.104394,NaN
3,5,OMVV.VI,2001-06-30T00:00:00Z,2001-06-30,2001-11-15,1704178457.61632,261593371.574251,199438283.407691,139758657.318887,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.153501,0.117029,0.082009,NaN
4,6,OMVV.VI,2001-09-30T00:00:00Z,2001-09-30,2001-11-08,1692966706.46458,247021992.14371,223213423.136865,133206040.77615,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.145911,0.131847,0.078682,NaN


In [16]:
meta_df.head()

,ticker,exchange_code,region_hq,country_hq,state_province_hq,founding_year,economic_sector,business_sector,industry_group,industry,activity
0,OMVV.VI,WBAH,Europe,Austria,WIEN,1956,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Refining and Marketing,Oil & Gas Refining and Marketing (NEC)
1,ENRA.KL,XKLS,Asia,Malaysia,WILAYAH PERSEKUTUAN,1992,Energy,Energy - Fossil Fuels,Oil & Gas,Integrated Oil & Gas,Integrated Oil & Gas
2,MDINp.TA,XTAE,Asia,Israel,<NA>,1992,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC)
3,CWV.V,XTSX,America,Canada,ALBERTA,2012,Energy,Energy - Fossil Fuels,Oil & Gas,Oil & Gas Exploration and Production,Oil & Gas Exploration and Production (NEC)
4,2178.HK,XHKG,Asia,China,GUANGDONG,2007,Energy,Energy - Fossil Fuels,Oil & Gas Related Equipment and Services,Oil & Gas Drilling,Oil & Gas Drilling (NEC)


In [ ]:
print(column_mapping)

{'Instrument': 'ticker', 'Date': 'date', 'Company Market Cap': 'market_cap', 'Price Close': 'close_price', 'Currency': 'currency', 'Period End Date': 'period_end_date', 'Balance Sheet Orig Announce Date': 'announce_date', 'Total Revenue': 'revenue', 'Gross Profit': 'gross_profit', 'EBITDA': 'ebitda', 'EBIT': 'ebit', 'Net Income after Tax': 'net_income', 'Total Assets': 'total_assets', 'Total Current Assets': 'total_current_assets', 'Total Liabilities': 'total_liabilites', 'Total Current Liabilities': 'total_current_liabilities', 'Long Term Debt Percentage of Total Assets': 'long_term_debt_p_assets', 'Short Term Debt Percentage of Total Assets': 'short_term_debt_p_assets', 'Free Cash Flow': 'fcf', 'Gross Profitp': 'gross_profit_p', 'EBITDAp': 'ebitda_p', 'EBITp': 'ebit_p', 'Net Income after Taxp': 'net_income_p', 'Exchange Market Identifier Code': 'exchange_code', 'Region of Headquarters': 'region_hq', 'Country of Headquarters': 'country_hq', 'State or Province of Headquarters': 'state_

In [14]:
fundamental_df = fundamental_df.rename(columns=column_mapping)
meta_df = meta_df.rename(columns=column_mapping)

full_data_fundamentals = fundamental_df[fundamental_df["ticker"].isin(full_data_companies)].drop(columns=["index"])
full_data_stock = stock_df[stock_df["ticker"].isin(full_data_companies)]
full_data_meta = meta_df[meta_df["ticker"].isin(full_data_companies)]

In [21]:
(
    full_data_stock.shape[0],
    full_data_fundamentals.count(axis=0, numeric_only=True).sum(),
    full_data_meta.count(axis=0, numeric_only=True).sum(),
)

(3206180, 621023, 861)

## Upload full data to WandB

In [22]:
put_dataset(full_data_fundamentals, filename="fundamentals-oil-final.feather", project="master-test")
put_dataset(full_data_stock, filename="stock-oil-final.feather", project="master-test")
put_dataset(full_data_meta, filename="meta-oil-final.feather", project="master-test")